# ResNet

In [1]:
class ResNetBasicBlock(nn.Module):
    def __init__(self,in_channels,out_channels,strides):
        super(ResNetBasicBlock,self).__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.conv2=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.stride=stride
    def forward(self,x):
        residual=x
        out=self.conv1(x)
        out=F.relu(self.bn1(out),inplace=True)
        out=self.conv2(out)
        out=self.bn2(out)
        out+=residual
        return F.relu(out)

NameError: name 'nn' is not defined

# 创建数据集

In [ ]:
data_transform=transforms.Compose([
    transforms.Resize((229,229)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_set=ImageFolder('../../catdog/train/',transforms=data_transform)
val_set=ImageFolder('../../catdog/valid/',transforms=data_transform)
classes=2

# 创建数据加载器

In [ ]:
train_loader=DataLoader(train_set,batch_size=32,shuffle=False,num_workers=0)
val_loader=DataLoader(val_set,batch_size=32,shuffle=False,num_workers=0)

# 创建模型

In [ ]:
my_resnet=resnet34(pretrained=True)
if is_cuda:
    my_resnet=my_resnet.cuda()
    
mu_resnet=nn.Squential(*list(my_resnet.children())[:-1])

for p in my_resnet.parameters():
    p.requires_grad=False


# 提取卷积特征

In [ ]:
trn_labels=[]
trn_features=[]
for d,la in train_loader:
    o=my_resnet(Variable(d.cuda()))
    o=o.view(size(0),-1)
    trn_labels.extend(la)
    trn_features.extend(o.cpu().data)
    
val_labels=[]
val_features=[]
for d,la in val_loader:
    o=my_resnet(Variable(d.cuda()))
    o=o.view(size(0),-1)
    val_labels.extend(la)
    val_features.extend(o.cpu().data)
    

# 为预卷积特征和加载器创建自定义的数据集

In [ ]:
class FeaturesDataset(Dataset):
    def __init__(self,fealist,lalist):
        self.fealist=fealist
        self.lalist=lalist
    def __getitem__(self,index):
        return (self.fealist[index],self.lablist[index])
    def __len__(self):
        return len(self.lablist)
    
trn_fea_set=FeaturesDataset(trn_features,trn_labels)
val_fea_set=FeaturesDataset(val_features,val_labels)

trn_fea_loader=DataLoader(trn_fea_set,batch_szie=64,shuffle=True)
val_fea_loader=DataLoader(val_fea_set,batch_szie=64)

# 创建简单线性模型

In [ ]:
class FullyConnecteModel(nn.Module):
    def __init__(self,in_size,out_size):
        super(FullyConnecteModel,self).__init__()
        self.fc=nn.Linear(in_size,out_size)
        
    def forward(self.inp):
        out=self.fc(inp)
        return out
    
fc_in_size=8192
fc=FullyConnecteModel(fc_in_size,classes)
if is_cuda:
    fc=fc.cuda()

# 训练并验证模型

In [ ]:
def fit(epoch,model,data_loader,phase='training',volatile=False,dataset_size=23000):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss=0.0
    running_correct=0
    for batch_idx,(data,target) in enumerate(data_loader):
        if is_cuda:
            data,target=data.cuda(),target.cuda()
        data,target=Variable(data,volatile),Variable(target)
        
        if phase =='training':
            optimizer.zero_grad()
            
        output=model(data)
        loss=loss_fn(output,target)
        running_loss+=loss_fn(output,target).data
        preds=output.data.max(dim=1,keepdim=True)[1]
        running_correct+=preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
            
    loss=running_loss/dataset_size
    accuracy=100.*running_correct/dataset_size
    print(f'{phase} loss is {loss} and {phase} accuracy is {running_correct}/{dataset_size}{accuracy}')
    return loss,accuracy

In [ ]:
train_losses,train_accuracy=[],[]
val_losses,val_accuracy=[],[]

for epoch in range(1,10):
    epoch_loss,epoch_accuracy=fit(epoch,fc,trn_fea_loader,phase='training')
    val_epoch_loss,val_epoch_accuracy=fit(epoch,fc,val_fea_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

# Inception

In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,**kwargs):
        super(BasicConv2d,self).__init__()
        self.conv=nn.Conv2d(in_channels,out_channels,bias=False,**kwargs)
        self.bn=nn.BatchNorm2d(out_channels)
    def forward(self,x):
        x=self.conv(x)
        x=self.bn(x)
        return F.relu(x,inplace=True)
    
class InceptionBasicBlock(nn.Module):
    super(InceptionBasicBlock,self).__init__()
    self.branch1x1=BasicConv2d(in_channels,64,kernel_size=1)
    
    self.branch5x5_1=BasicConv2d(in_channels,48,kernel_size=1)
    self.branch5x5_2=BasicConv2d(48,64,kernel_size=5,padding=2)
    
    self.branch3x3db1_1=BasicConv2d(in_channels,64,kernel_size=1)
    self.branch3x3db1_2=BasicConv2d(64,96,kernel_size=3,padding=1)   
    
    self.branch_pool=BasicConv2d(in_channels,pool_features,kernel_size=1)
    
    def forward(self,x):
        branch1x1=self.branch1x1(x)
        
        branch5x5=self.branch5x5_1(x)
        branch5x5=self.branch5x5_2(branch5x5)   
        
        branch3x3db1=self.branch3x3db1_1(x)
        branch3x3db1=self.branch3x3db1_2(branch3x3db1)
        
        branch_pool=F.avg_pool2d(x,kernel_size=3,stride=1,padding=1)
        branch_pool=self.branch_pool(branch_pool)
        
        outputs=[branch1x1,branch5x5,branch3x3db1,branch_pool]
        return torch.cat(outputs,1)



# DenseNet

In [ ]:
class DenseBlock(nn.Sequential):
    def __init__(self,num_layers,num_input_features,bn_size,growth_rate,drop_rate):
        super(DenseBlock,self).__init__()
        for i in range(num_layers):
            layer=DenseLayer(num_input_features+i*growth_rate,growth_rate,bn_size,drop_rate)
            self.add_module('denselayer%d' % (i+1),layer)
            
class DenseLayer(nn.Sequential):
    def __init__(self,num_input_features,growth_rate,bn_size,drop_rate):
        super(DenseLayer,self).__init__()
        self.add_module('norm.1',nn.BatchNorm2d(num_input_features)),
        self.add_module('relu.1',nn.ReLU(inplace=True)),
        self.add_module('conv.1',nn.Conv2d(num_input_features,bn_size*growth_rate,kernel_size=3,stride=1,padding=1,bias=False)),
        
        
        self.add_module('norm.2',nn.BatchNorm2d(bn_size*growth_rate)),
        self.add_module('relu.2',nn.ReLU(inplace=True)),
        self.add_module('conv.2',nn.Conv2d(bn_size*growth_rate,bn_size*growth_rate,kernel_size=3,stride=1,padding=1,bias=False)),
        
        self.drop_rate=drop_rate
        
    def forward(self,x):
        new_features=super(DenseLayer,self).forward(x)
        if self.drop_rate>0:
            new_features=F.dropout(new_features,p=self.drop_rate,training=self.training)
        return torch.cat([x,new_features],1)
            

# 模型集成：resnet+inception+densenet

##  创建三个模型

In [ ]:
my_resnet=resnet34(pretrained=True)
if is_cuda:
    my_resnet=my_resnet.cuda()
    
my_resnet=nn.Sequential(*list(my_resnet.children())[:-1])

for p in my_resnet.parameters():
    p.requires_grad=False
    
    
    
my_inception=inception_v3(pretrained=True)
if is_cuda:
    my_inception=my_inception.cuda()
    
for p in my_inception.parameters():
    p.requires_grad=False
    
    
my_densenet=densenet121(pretrained=True)
if is_cuda:
    my_densenet=my_densenet.cuda()

for p in my_densenet.parameters():
    p.requires_grad=False   
    

## 分别提取特征

In [ ]:
trn_labels=[]
trn_resnet_features=[]
for d,la in train_loader:
    o=my_resnet(Variable(d.cuda()))
    o=o.view(size(0),-1)
    trn_labels.extend(la)
    trn_resnet_features.extend(o.cpu().data)
    
val_labels=[]
val_resnet_features=[]
for d,la in val_loader:
    o=my_resnet(Variable(d.cuda()))
    o=o.view(size(0),-1)
    val_labels.extend(la)
    val_resnet_features.extend(o.cpu().data)
    
    
    
trn_inception_features=LayerActivations(my_inception.Mixed_7c)
for da,la in train_loader:
    _=my_inception(Variable(da.cuda()))
    
trn_inception_features.remove()

val_inception_features=LayerActivations(my_inception.Mixed_7c)
for da,la in val_loader:
    _=my_inception(Variable(da.cuda()))
    
val_inception_features.remove()



trn_densenet_features=[]
for d,la in train_loader:
    o=my_densenet(Variable(d.cuda()))
    o=o.view(size(0),-1)
    trn_densenet_features.extend(o.cpu().data)
    
val_densenet_features=[]
for d,la in val_loader:
    o=my_densenet(Variable(d.cuda()))
    o=o.view(size(0),-1)
    val_densenet_features.extend(o.cpu().data)

## 创建自定义数据集和数据加载器

In [ ]:
class FeaturesDataset(Dataset):
    def __init__(self,fealist1,fealist2,fealist2,lablist):
        self.fealist1=fealist1
        self.fealist2=fealist2
        self.fealist3=fealist3
        self.lablist=lablist
    def __getitem__(self,index):
        return (self.fealist1[index],self.fealist2[index],self.fealist3[index],self.lablist[index])
    def __len__(self):
        return len(self.lablist)
    
trn_feat_set=FeaturesDataset(trn_resnet_features,trn_inception_features,trn_densenet_features,trn_labels)
val_feat_set=FeaturesDataset(val_resnet_features,val_inception_features,val_densenet_features,val_labels)

trn_feat_loader=DataLoader(trn_feat_set,batch_size=64,shuffle=True)
val_feat_loader=DataLoader(val_feat_set,batch_size=64,shuffle=True)

## 创建集成模型

In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self,out_size,training=True):
        super(EnsembleModel,self).__init__()
        self.fc1=nn.Linear(8192,512)
        self.fc2=nn.Linear(131072,512)       
        self.fc3=nn.Linear(82944,512)    
        
        self.fc4=nn.Linear(512,out_size)      
    def foeward(self,inp1,inp2,inp3):
        out1=self.fc1(inp1)
        out2=self.fc2(inp2)
        out3=self.fc3(inp3)
        out=pit1+out2+out3
        out=self.fc4(F.dropout(out,training=self.training))
        return out
    
em=EnsembleModel(2)
if is_cuda:
    em=em.cuda()


## 训练和验证模型

In [ ]:
def fit(epoch,model,data_loader,phase='training',volatile=False,dataset_size=23000):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss=0.0
    running_correct=0
    for batch_idx,(data1,data2,data3,target) in enumerate(data_loader):
        if is_cuda:
            data1,data2,data3,target=data1.cuda(),data2.cuda(),data3.cuda(),target.cuda()
        data1,data2,data3,target=Variable(data1,volatile),Variable(data2,volatile),Variable(data3,volatile),Variable(target)
        
        if phase =='training':
            optimizer.zero_grad()
            
        output=model(data1,data2,data3)
        loss=loss_fn(output,target)
        running_loss+=loss_fn(output,target).data
        preds=output.data.max(dim=1,keepdim=True)[1]
        running_correct+=preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
            
    loss=running_loss/dataset_size
    accuracy=100.*running_correct/dataset_size
    print(f'{phase} loss is {loss} and {phase} accuracy is {running_correct}/{dataset_size}{accuracy}')
    return loss,accuracy

In [ ]:
train_losses,train_accuracy=[],[]
val_losses,val_accuracy=[],[]

for epoch in range(1,10):
    epoch_loss,epoch_accuracy=fit(epoch,em,trn_fea_loader,phase='training')
    val_epoch_loss,val_epoch_accuracy=fit(epoch,em,val_fea_loader,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)